In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch < 15:
        return 0.0005
    elif epoch < 30:
        return 0.0001
    elif epoch < 45:
        return 0.00005
    else:
        return 0.00001

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

     # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [5]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.75366, saving model to best_m.h5
 - 8s - loss: 0.5844 - acc: 0.7241 - val_loss: 0.7537 - val_acc: 0.4663
Epoch 2/100
Epoch 00002: val_loss improved from 0.75366 to 0.68708, saving model to best_m.h5
 - 4s - loss: 0.3887 - acc: 0.8225 - val_loss: 0.6871 - val_acc: 0.6733
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.3783 - acc: 0.8195 - val_loss: 1.7931 - val_acc: 0.5960
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.3427 - acc: 0.8258 - val_loss: 1.5125 - val_acc: 0.6359
Epoch 5/100
Epoch 00005: val_loss improved from 0.68708 to 0.52680, saving model to best_m.h5
 - 4s - loss: 0.4256 - acc: 0.8322 - val_loss: 0.5268 - val_acc: 0.8030
Epoch 6/100
Epoch 00006: val_loss improved from 0.52680 to 0.51448, saving model to best_m.h5
 - 4s - loss: 0.3210 - acc: 0.8566 - val_loss: 0.5145 - val_acc: 0.7656
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3236 - acc: 0.8466 - val_loss:

Epoch 23/100
Epoch 00023: val_loss did not improve
 - 4s - loss: 0.2656 - acc: 0.8895 - val_loss: 0.2161 - val_acc: 0.9052
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2194 - acc: 0.9133 - val_loss: 0.2210 - val_acc: 0.9177
Epoch 25/100
Epoch 00025: val_loss improved from 0.20201 to 0.19894, saving model to best_m.h5
 - 4s - loss: 0.2483 - acc: 0.8983 - val_loss: 0.1989 - val_acc: 0.9227
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2444 - acc: 0.8931 - val_loss: 0.2067 - val_acc: 0.9102
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2480 - acc: 0.8917 - val_loss: 0.1991 - val_acc: 0.9177
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2544 - acc: 0.8995 - val_loss: 0.2065 - val_acc: 0.9002
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2176 - acc: 0.9117 - val_loss: 0.2117 - val_acc: 0.9177
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2325 - acc: 0.9031 - val_los

Epoch 86/100
Epoch 00086: val_loss did not improve
 - 4s - loss: 0.2115 - acc: 0.9176 - val_loss: 0.1817 - val_acc: 0.9127
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 4s - loss: 0.1572 - acc: 0.9292 - val_loss: 0.1800 - val_acc: 0.9202
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 4s - loss: 0.1747 - acc: 0.9275 - val_loss: 0.1769 - val_acc: 0.9152
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 4s - loss: 0.1677 - acc: 0.9292 - val_loss: 0.1784 - val_acc: 0.9252
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 4s - loss: 0.1840 - acc: 0.9141 - val_loss: 0.1777 - val_acc: 0.9177
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 4s - loss: 0.1766 - acc: 0.9308 - val_loss: 0.1782 - val_acc: 0.9252
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 4s - loss: 0.1828 - acc: 0.9181 - val_loss: 0.1829 - val_acc: 0.9252
Epoch 93/100
Epoch 00093: val_loss improved from 0.17576 to 0.17562, saving model to best_m.h5
 - 4s - loss: 0.1560 - acc: 0.9367 - val_los

Epoch 48/100
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.1981 - acc: 0.9281 - val_loss: 0.1940 - val_acc: 0.9252
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.2036 - acc: 0.9073 - val_loss: 0.1947 - val_acc: 0.9252
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.1684 - acc: 0.9267 - val_loss: 0.1916 - val_acc: 0.9277
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.1855 - acc: 0.9214 - val_loss: 0.1925 - val_acc: 0.9277
Epoch 52/100
Epoch 00052: val_loss improved from 0.19149 to 0.19091, saving model to best_m.h5
 - 4s - loss: 0.1924 - acc: 0.9150 - val_loss: 0.1909 - val_acc: 0.9277
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 4s - loss: 0.1735 - acc: 0.9333 - val_loss: 0.1931 - val_acc: 0.9277
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 4s - loss: 0.1692 - acc: 0.9267 - val_loss: 0.1932 - val_acc: 0.9277
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 4s - loss: 0.1854 - acc: 0.9217 - val_los

Epoch 39/100
Epoch 00039: val_loss did not improve
 - 4s - loss: 0.2190 - acc: 0.9070 - val_loss: 0.1718 - val_acc: 0.9252
Epoch 40/100
Epoch 00040: val_loss did not improve
 - 4s - loss: 0.1994 - acc: 0.9233 - val_loss: 0.1698 - val_acc: 0.9352
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 4s - loss: 0.1935 - acc: 0.9208 - val_loss: 0.1938 - val_acc: 0.9152
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 4s - loss: 0.2098 - acc: 0.9189 - val_loss: 0.1719 - val_acc: 0.9352
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 4s - loss: 0.2184 - acc: 0.9075 - val_loss: 0.1861 - val_acc: 0.9352
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 4s - loss: 0.2180 - acc: 0.9043 - val_loss: 0.1778 - val_acc: 0.9377
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 4s - loss: 0.2153 - acc: 0.9125 - val_loss: 0.1830 - val_acc: 0.9277
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 4s - loss: 0.1981 - acc: 0.9150 - val_loss: 0.1702 - val_acc: 0.9302
Epoch 47/100
Epo

In [6]:
with open('../features/incept_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug3_sub.csv', index=False)
# deep2 0.2110
# deep3 0.2119

# deep3 new 2011

0.201199186747
         id  is_iceberg
0  5941774d    0.075037
1  4023181e    0.875867
2  b20200e4    0.056142
3  e7f018bb    0.998137
4  4371c8c3    0.311983


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70379, saving model to best_m.h5
 - 26s - loss: 1.2550 - acc: 0.5603 - val_loss: 0.7038 - val_acc: 0.4663
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5912 - acc: 0.6662 - val_loss: 0.7226 - val_acc: 0.4663
Epoch 3/100
Epoch 00003: val_loss improved from 0.70379 to 0.66071, saving model to best_m.h5
 - 6s - loss: 0.5339 - acc: 0.7733 - val_loss: 0.6607 - val_acc: 0.5786
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4741 - acc: 0.8183 - val_loss: 0.6636 - val_acc: 0.5985
Epoch 5/100
Epoch 00005: val_loss improved from 0.66071 to 0.49273, saving model to best_m.h5
 - 6s - loss: 0.4396 - acc: 0.8116 - val_loss: 0.4927 - val_acc: 0.7855
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.4372 - acc: 0.8125 - val_loss: 0.5870 - val_acc: 0.7756
Epoch 7/100
Epoch 00007: val_loss improved from 0.49273 to 0.46280, saving model to best_m.h5
 - 6s - loss: 0.4488 - acc: 0.8164 - val_loss

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 6s - loss: 0.2626 - acc: 0.9148 - val_loss: 0.3339 - val_acc: 0.8529
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 6s - loss: 0.2416 - acc: 0.9106 - val_loss: 0.3304 - val_acc: 0.8579
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 6s - loss: 0.2279 - acc: 0.9192 - val_loss: 0.3359 - val_acc: 0.8529
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 6s - loss: 0.2665 - acc: 0.9058 - val_loss: 0.3328 - val_acc: 0.8529
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 6s - loss: 0.2531 - acc: 0.9008 - val_loss: 0.3236 - val_acc: 0.8703
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.2600 - acc: 0.8933 - val_loss: 0.3271 - val_acc: 0.8678
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.2439 - acc: 0.9073 - val_loss: 0.3279 - val_acc: 0.8628
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.2435 - acc: 0.9058 - val_loss: 0.3288 - val_acc: 0.8529
Epoch 70/100
Epo

Epoch 24/100
Epoch 00024: val_loss did not improve
 - 6s - loss: 0.3094 - acc: 0.8673 - val_loss: 0.2697 - val_acc: 0.8903
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 6s - loss: 0.3216 - acc: 0.8554 - val_loss: 0.3294 - val_acc: 0.8653
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 6s - loss: 0.3223 - acc: 0.8617 - val_loss: 0.2415 - val_acc: 0.8878
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 6s - loss: 0.3187 - acc: 0.8545 - val_loss: 0.2329 - val_acc: 0.9177
Epoch 28/100
Epoch 00028: val_loss improved from 0.21912 to 0.21131, saving model to best_m.h5
 - 6s - loss: 0.3078 - acc: 0.8641 - val_loss: 0.2113 - val_acc: 0.9152
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 6s - loss: 0.2931 - acc: 0.8783 - val_loss: 0.2262 - val_acc: 0.8878
Epoch 30/100
Epoch 00030: val_loss improved from 0.21131 to 0.20529, saving model to best_m.h5
 - 6s - loss: 0.2785 - acc: 0.8800 - val_loss: 0.2053 - val_acc: 0.9102
Epoch 31/100
Epoch 00031: val_loss did not improve


Epoch 29/100
Epoch 00029: val_loss did not improve
 - 6s - loss: 0.2567 - acc: 0.8931 - val_loss: 0.2556 - val_acc: 0.8928
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 6s - loss: 0.3015 - acc: 0.8739 - val_loss: 0.2793 - val_acc: 0.8928
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 6s - loss: 0.2771 - acc: 0.8791 - val_loss: 0.2528 - val_acc: 0.9127
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 6s - loss: 0.2824 - acc: 0.8808 - val_loss: 0.2722 - val_acc: 0.8928
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 6s - loss: 0.3028 - acc: 0.8700 - val_loss: 0.2652 - val_acc: 0.8828
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 6s - loss: 0.2502 - acc: 0.8950 - val_loss: 0.2684 - val_acc: 0.8828
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 6s - loss: 0.2781 - acc: 0.8733 - val_loss: 0.2612 - val_acc: 0.8878
Epoch 36/100
Epoch 00036: val_loss did not improve
 - 6s - loss: 0.2537 - acc: 0.8942 - val_loss: 0.2578 - val_acc: 0.9052
Epoch 37/100
Epo

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 6s - loss: 0.3038 - acc: 0.8558 - val_loss: 0.2207 - val_acc: 0.9102
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 6s - loss: 0.3035 - acc: 0.8533 - val_loss: 0.2395 - val_acc: 0.9077
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 6s - loss: 0.3629 - acc: 0.8547 - val_loss: 0.2319 - val_acc: 0.9152
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 6s - loss: 0.3023 - acc: 0.8591 - val_loss: 0.2194 - val_acc: 0.9177
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 6s - loss: 0.3193 - acc: 0.8525 - val_loss: 0.2226 - val_acc: 0.9202
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 6s - loss: 0.2838 - acc: 0.8758 - val_loss: 0.2163 - val_acc: 0.9152
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 6s - loss: 0.3091 - acc: 0.8656 - val_loss: 0.2235 - val_acc: 0.9202
Epoch 35/100
Epoch 00035: val_loss improved from 0.21479 to 0.21328, saving model to best_m.h5
 - 6s - loss: 0.2923 - acc: 0.8667 - val_los

In [9]:
with open('../features/resnet_aug3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug3_sub.csv', index=False)
# deep2 0.227
# deep3 0.223
# deep3 new

0.230381632957
         id  is_iceberg
0  5941774d    0.212729
1  4023181e    0.731043
2  b20200e4    0.044830
3  e7f018bb    0.958758
4  4371c8c3    0.095437
